# 02 - Clasificación con Keras

Vamos a entrenar un modelo de Keras para clasificar imagenes de ropa.

Usaremos el conjunto de datos de [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist)
que contiene mas de 70.000 imagenes en 10 categorias. Las imagenes muestran articulos individuales de ropa a una resolución baja (28x28 pixeles) como se ve en la siguiente imagen:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST esta construido como un reemplazo para el conjunto de datos clasico [MNIST](http://yann.lecun.com/exdb/mnist/) 
casi siempre utilizado como el "Hola Mundo" de los sistemas de visión por computadora. El conjunto de datos de MNIST contiene imágenes de dígitos escrito a mano (0, 1, 2, etc.) en un formato idéntico al de los artículos de ropa que vamos a utilizar.


Las imagenes son arrays de numpy de 28x28 pixeles, con valores comprendidos entre 0 y 255, los valores de los pixeles. Las etiquetas de integros, que van del 0 al 9. Estos corresponden a la *class* de ropa que la imagen representa.

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

Cada imagen es mapeada a una unica etiqueta. 

In [ ]:
# librerias

import warnings
warnings.filterwarnings('ignore')

from keras.datasets import fashion_mnist

from keras import Sequential, Input
from keras.layers import Flatten, Dense

import numpy as np

import pylab as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import classification_report as report

In [ ]:
# etiquetas

label_names = {0: 'T-shirt/top', 
               1: 'Trouser', 
               2: 'Pullover', 
               3: 'Dress', 
               4: 'Coat',
               5: 'Sandal', 
               6: 'Shirt', 
               7: 'Sneaker', 
               8: 'Bag', 
               9: 'Ankle boot'}

**Caraga de datos**

In [ ]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
# 60k imagenes de 28x28 pixeles para entrenar y 10k para testear con sus etiquetas

X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Exploración de los datos

In [ ]:
# valores entre 0 y 255

plt.imshow(X_train[29000])
plt.colorbar();

Los datos de entrada en una red neuronal tienen que ser normalizados. En este caso, dividir entre el máximo (255)

In [ ]:
X_train = X_train/255.

X_test = X_test/255.

Para verificar que el conjunto de datos esta en el formato adecuado y que estan listos para construir y entrenar la red, vamos a ver las primeras 25 imagenes con su nombre.

In [ ]:
plt.figure(figsize=(10,10))  # figura 100x100


for i in range(25):   # 25 imagenes
    
    plt.subplot(5,5,i+1)
    
    plt.xticks([])
    plt.yticks([])
    
    plt.grid(False)
    
    plt.imshow(X_train[i], cmap=plt.cm.binary)
    
    plt.xlabel(label_names[y_train[i]])
     
    
plt.show();

### Modelo

Construir la red neuronal requiere configurar las capas del modelo y luego compilar el modelo. 

Los bloques de construccion básicos de una red neuronal son las *capas*. La función de las capas es extraer la características fundamentales de las imágenes para poder luego clasificarlas. La mayoria de las capas como `Dense`, tienen parametros que son aprendidos durante el entrenamiento.

La primera capa de esta red, `Flatten`, 
transforma el formato de las imágenes de un array bidimensional (28x28 pixeles) a un array unidimensional (28x28 pixeles = 784 pixeles). Se le llama "aplanado" porque pasamos de una matriz de 28x28 a un vector de 784 elemento.

Después de que los pixeles sean "aplanados", la secuencia consiste de dos capas `Dense`. También se llama a estas capa `Full connected`, pues está todo conectado con todo. La primera capa `Dense` tiene 128 nodos o neuronas. La segunda capa es una capa de 10 nodos *softmax* (regresión logística) que devuelve un array de 10 probabilidades que suman a 1. Cada nodo contiene una calificacion que indica la probabilidad que la actual imagen pertenece a una de las 10 clases.

In [ ]:
model = Sequential()

model.add(Flatten(input_shape=(28, 28)))

model.add(Dense(128, activation='relu'))

model.add(Dense(10, activation='softmax'))

Antes de que el modelo este listo para entrenar, se necesitan algunas configuraciones más. Estas son agregadas durante el paso de compilación del modelo:

* *Loss function* — Función de coste o pérdida. Funcióin a minimizar para el ajuste.

* *Optimizer* — Algoritmo de optimización.

* *Metrics* — Métrica para el ajuste.

El siguiente ejemplo usa *accuracy* (acierto) , la proporción de imágenes que son correctamente clasificadas.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

### Entrenamiento

In [ ]:
model.fit(X_train, y_train, epochs=10)

### Evaluación y predicción

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
y_prob = model.predict(X_test)

In [ ]:
y_prob[0]

In [ ]:
plt.bar(label_names.values() ,y_prob[600])
plt.xticks(rotation=45);

In [ ]:
sum(y_prob[10])

In [ ]:
np.argmax(y_prob[0])

In [ ]:
label_names[np.argmax(y_prob[0])]

In [ ]:
y_pred = np.argmax(y_prob, axis=1)

y_pred

In [ ]:
y_test

In [ ]:
print(report(y_test, y_pred))

In [ ]:
plt.figure(figsize=(15, 8))

ax=sns.heatmap(cm(y_test, y_pred)/cm(y_test, y_pred).sum(), annot=True)

plt.title('Matriz confusion')
plt.ylabel('Verdad')
plt.xlabel('Prediccion')
plt.show();

In [ ]:
label_names

### Guardado y carga del modelo

In [ ]:
model.save_weights('model/MNIST_weights.h5')    # pesos de la red

In [ ]:
model.load_weights('model/MNIST_weights.h5')

In [ ]:
model.save('model/MNIST_Model.h5')              # modelo completo

In [ ]:
from keras.models import load_model

In [ ]:
modelo_nuevo = load_model('model/MNIST_Model.h5')

In [ ]:
modelo_nuevo.predict(X_test)[0]